# Dashboard Finanças TecnoJR

## Importação das bibliotecas necessárias

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import locale

## Importação dos dados

In [2]:
df = pd.read_excel('../data/livro_caixa.xlsx')
df.head()

,Data,Nome,Informações do Pgt.,Descrição,Classificação,Valor,Fluxo de caixa
0,2023-01-05,Adv Junior Consultoria,Boleto,Parcela 8 de 12,Taxa de Serviço*,250.0,Saída
1,2023-01-13,Banco do Bradesco S.A,Débito,Taxação de Manutenção BBDE,Taxa de Serviço*,121.9,Saída
2,2023-02-07,Adv Junior Consultoria,Pix,Parcela 9 de 12,Taxa de Serviço*,250.0,Saída
3,2023-02-15,Banco do Bradesco S.A,Débito,Taxação de Manutenção BBDE,Taxa de Serviço*,121.9,Saída
4,2023-02-16,Evron,Pix,Parcela 2 de 4,Projeto,960.0,Entrada


## total de entradas, saídas e caíxa em 2023

### Entradas

In [3]:
total_entradas = df[df['Fluxo de caixa'] == 'Entrada']['Valor'].sum()
print(f"Total de entrada em 2023: R$ {total_entradas}")

Total de entrada em 2023: R$ 14407.0


### Saídas

In [4]:
total_saidas = df[df['Fluxo de caixa'] == 'Saída']['Valor'].sum()
print(f"Total de saída em 2023: R$ {total_saidas}")

Total de saída em 2023: R$ 15258.05


### Caixa

In [5]:
saldo_anterior = 1702.32
caixa = total_entradas - total_saidas + saldo_anterior
print(f"Caixa em 2023: R$ {round(caixa, 2)}")

Caixa em 2023: R$ 851.27


## Total de entradas, saídas e caixa por mês em 2023

In [6]:
df_grouped_by_month = df.groupby([df['Data'].dt.month, 'Fluxo de caixa']).agg({'Valor': 'sum'}).reset_index()
df_grouped_by_month['Valor'] = df_grouped_by_month['Valor'].apply(lambda x: round(x, 2))

novas_linhas = [{'Data': 1, 'Fluxo de caixa': 'Entrada', 'Valor': 0},
                {'Data': 5, 'Fluxo de caixa': 'Entrada', 'Valor': 0}]
novo_df = pd.DataFrame(novas_linhas)

df_grouped_by_month = pd.concat([df_grouped_by_month, novo_df],
                                ignore_index=True).sort_values(by=['Data']).reset_index(drop=True)

meses = {
    1: 'Janeiro',
    2: 'Fevereiro',
    3: 'Março',
    4: 'Abril',
    5: 'Maio',
    6: 'Junho',
    7: 'Julho',
    8: 'Agosto',
    9: 'Setembro',
    10: 'Outubro',
    11: 'Novembro',
    12: 'Dezembro'
}

df_grouped_by_month['Data'] = df_grouped_by_month['Data'].apply(lambda x: meses[x])
fig = px.bar(df_grouped_by_month, x='Data', y='Valor', color='Fluxo de caixa', barmode='group',
            title='Fluxo de caixa por mês', text_auto='.2s', color_discrete_sequence=['#00CC96', '#EF553B'])
fig.update_xaxes(title_text="Mês")
fig.update_yaxes(title_text="Valor (R$)")
fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
fig.update_layout(
    title_x=0.4,  # Centraliza o título horizontalmente
    title_font=dict(size=32)  # Define o tamanho da fonte do título
)

## Entradas e saídas em um mês especifico

In [12]:
def format_moeda(valor):
    locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')
    valor = locale.currency(valor, grouping=True, symbol=None)
    return valor

selected_month = 'Maio'
df_selected_month = df[df['Data'].dt.month == list(meses.keys())[list(meses.values()).index(selected_month)]]
df_selected_month.loc[::, 'Data'] = df_selected_month['Data'].dt.day
df_selected_month.loc[::, 'Valor'] = df_selected_month['Valor'].apply(lambda x: f"R$ {format_moeda(x)}")

selected_columns = ['Dia', 'Nome', 'Descrição', 'Classificação', 'Valor', 'Fluxo de caixa']

texto_cor = 'white'
fundo_cor = 'rgb(31, 31, 31)'
borda_cor = 'white'

fig = go.Figure(data=[go.Table(
    header=dict(values=selected_columns,
                fill_color=fundo_cor,
                font=dict(color=texto_cor),
                align='left'),
    cells=dict(values=[df_selected_month['Data'], df_selected_month['Nome'],
                    df_selected_month['Descrição'], df_selected_month['Classificação'],
                    df_selected_month['Valor'], df_selected_month['Fluxo de caixa']],
                fill_color=fundo_cor, 
                font=dict(color=texto_cor),
                align='left'))
])

fig.update_layout(
    title=f'Fluxo de caixa em {selected_month}',
    title_x=0.5,
    title_font=dict(size=32),
    paper_bgcolor=fundo_cor,
    plot_bgcolor=fundo_cor,
    margin=dict(l=0, r=0, t=50, b=0),
    font=dict(color=texto_cor),
    height=600
)

fig.show()